In [1]:
import tensorflow as tf
import json
# import tflearn
from tensorflow import keras
from tensorflow.keras import layers
import nltk
import random
import numpy as np

In [ ]:
print(tf.__version__)

2.6.0


In [2]:
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
with open('intents.json') as data:
    intents = json.load(data)

In [4]:
with open('/content/drive/MyDrive/ChatBot - Vighnesh/intents.json') as data:
    intents = json.load(data)

In [5]:
intents

{'intents': [{'patterns': ['Hi', 'Hey', "What's up", 'Hello'],
   'responses': ['Hello', 'Hi', 'Hey'],
   'tag': 'greeting'},
  {'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again'],
   'tag': 'goodbye'},
  {'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"],
   'tag': 'thanks'},
  {'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ["I'm Chadbot, here to help", 'My name is Chadbot'],
   'tag': 'about'},
  {'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Chadbot.', 'My name is Chadbot', 'Chadbot'],
   'tag': 'name'},
  {'patterns': ['Could you help me?',
    'give me a hand please',
    'Can you help?',
    'What can you do for me?',
    'I need a support',
    'I need a help',

In [6]:
words = []
documents = []
tags = []
ignore = ['?']
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in tags:
            tags.append(intent['tag'])

In [7]:
print(documents)

[(['Hi'], 'greeting'), (['Hey'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['Hello'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Thanks', 'for', 'the', 'help'], 'thanks'), (['Who', 'are', 'you', '?'], 'about'), (['What', 'are', 'you', '?'], 'about'), (['Who', 'you', 'are', '?'], 'about'), (['what', 'is', 'your', 'name'], 'name'), (['what', 'should', 'I', 'call', 'you'], 'name'), (['whats', 'your', 'name', '?'], 'name'), (['Could', 'you', 'help', 'me', '?'], 'help'), (['give', 'me', 'a', 'hand', 'please'], 'help'), (['Can', 'you', 'help', '?'], 'help'), (['What', 'can', 'you', 'do', 'for', 'me', '?'], 'help'), (['I', 'need', 'a', 'support'], 'help'), (['I', 'need', 'a', 'help'], 'help'), (['support', 'me', 'please'], 'help'), (['I', 'am', 'looking', 'for', 'a', 'computer'], 'computer'), (['how', 'to', 'select', 'a', 'good', 'comp

In [8]:
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

tags = sorted(list(set(tags)))

print(len(documents), "documents")
print(len(tags), "tags", tags)
print(len(words), "unique words", words)

32 documents
8 tags ['about', 'computer', 'goodbye', 'greeting', 'help', 'name', 'phone', 'thanks']
53 unique words ["'m", "'s", 'a', 'about', 'am', 'any', 'ar', 'buy', 'bye', 'cal', 'can', 'comput', 'could', 'do', 'for', 'giv', 'good', 'goodby', 'hand', 'hav', 'hello', 'help', 'her', 'hey', 'hi', 'how', 'i', 'is', 'lat', 'learn', 'look', 'me', 'mobl', 'nam', 'nee', 'phon', 'pleas', 'recommend', 'see', 'select', 'should', 'som', 'support', 'thank', 'that', 'the', 'to', 'up', 'want', 'what', 'who', 'yo', 'you']


In [9]:
training = []
output = []
output_empty = [0]*len(tags)
#print(output_empty)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[tags.index(doc[1])] = 1
        
    training.append([bag, output_row])

    
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [10]:
train_x=np.array(train_x, dtype='float')
train_x.shape

(32, 53)

In [11]:
train_y=np.array(train_y, dtype='float')
train_y.shape

(32, 8)

In [13]:
model = keras.Sequential([
  layers.Dense(128, input_shape=train_x.shape[1:], activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(64, activation='relu'),
  layers.Dropout(0.5),
  layers.Dense(len(tags), activation='softmax')                     
])

In [14]:
model.compile(optimizer = 'adam', loss = 'mae', metrics = ['accuracy'])

In [15]:
model.fit(train_x, train_y, batch_size = 4, epochs = 100)

Epoch 1/100
8/8 [==============================] - 1s 3ms/step - loss: 0.2186 - accuracy: 0.0312
Epoch 2/100
8/8 [==============================] - 0s 2ms/step - loss: 0.2187 - accuracy: 0.1250
Epoch 3/100
8/8 [==============================] - 0s 2ms/step - loss: 0.2191 - accuracy: 0.1250
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: 0.2133 - accuracy: 0.1875
Epoch 5/100
8/8 [==============================] - 0s 2ms/step - loss: 0.2150 - accuracy: 0.2188
Epoch 6/100
8/8 [==============================] - 0s 2ms/step - loss: 0.2105 - accuracy: 0.3438
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 0.2063 - accuracy: 0.2812
Epoch 8/100
8/8 [==============================] - 0s 4ms/step - loss: 0.2093 - accuracy: 0.2500
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: 0.2045 - accuracy: 0.2812
Epoch 10/100
8/8 [==============================] - 0s 2ms/step - loss: 0.2009 - accuracy: 0.5000
Epoch 11/100
8/8 [===========

In [ ]:
model.save('model.h5')

In [ ]:
import pickle
pickle.dump({'words':words, 'tags':tags, 'train_x':train_x, 'train_y':train_y}, open("training_data", "wb"))

In [ ]:
data = pickle.load(open("training_data", "rb"))
words = data['words']
tags = data['tags']
train_x = data['train_x']
train_y = data['train_y']

In [ ]:
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
from tensorflow.keras.models import load_model
loaded_model = load_model('model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
def sentence_cleanup(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence, words, show_details=False):
    sentence_words = sentence_cleanup(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w==s:
                bag[i]=1
                if show_details:
                    print('Found in bag: %s' %w)
    return (np.array(bag))                

In [29]:
def classify(sentence):
    bag = bag_of_words(sentence, words)
    results = model.predict(bag.reshape(-1, len(words)))
    return np.argmax(results)

def response(sentence, userID='123', show_details=False):
    predicted_class=classify(sentence)
    predicted_tag = tags[predicted_class]
    for i in intents['intents']:
      if i['tag']==predicted_tag:
        return print(random.choice(i['responses']))

In [30]:
classify('Who are you?')

0

In [28]:
response('Who are you?')

I'm Chadbot, here to help


In [23]:
bag = bag_of_words("Hey there", words)
print(bag.shape)
results = model.predict(bag.reshape(-1, 53))
print(np.argmax(results))

(53,)
3


In [22]:
print(tags)

['about', 'computer', 'goodbye', 'greeting', 'help', 'name', 'phone', 'thanks']
